-----------
스크립트프로그래밍 2017136122 조윤성

------------

In [3]:
import sys
sys.setrecursionlimit(10000)

class TicTacToe:
    def __init__(self):
        self.N = 3
        self.map = [['E' for _ in range(self.N)] for _ in range(self.N)]    # E: 빈 공간(Empty)
        self.map_index_description = [h*self.N + w for h in range(self.N) for w in range(self.N)]
        self.player_types = ('X', 'O')  # 선공: X, 후공: O
        self.global_step = 0

        self.win_reward = 1.0
        self.defeat_reward = -1.0
        self.draw_reward = 0.0
        self.player_result = {'X': self.draw_reward, 'O': self.draw_reward}

        self.done = False

    def reset(self):
        self.map = [['E' for _ in range(self.N)] for _ in range(self.N)]
        self.global_step = 0
        self.player_result = {'X': self.draw_reward, 'O': self.draw_reward}
        self.done = False

        return self.map

    def step(self, action):
        action_coord_h, action_coord_w = self.transform_action(action)
        if self.global_step % 2 == 0:
            current_player_idx = 0
            other_player_idx = 1
        else:
            current_player_idx = 1
            other_player_idx = 0
        current_player_type = self.player_types[current_player_idx]
        other_player_type = self.player_types[other_player_idx]
        if self.map[action_coord_h][action_coord_w] == 'E':
            self.map[action_coord_h][action_coord_w] = current_player_type
            if self.is_win(current_player_type):    # 현재 플레이어 승리
                self.player_result[current_player_type] = self.win_reward
                self.player_result[other_player_type] = self.defeat_reward
                self.done = True
            elif self.is_full():    # 무승부
                self.done = True
            else:
                pass
        else:   # 현재 플레이어 패배
            self.player_result[current_player_type] = self.defeat_reward
            self.player_result[other_player_type] = self.win_reward
            self.done = True
        self.global_step += 1

        return self.map, self.player_result, self.done

    def transform_action(self, action):
        return divmod(action, self.N)

    def is_win(self, current_player_type):
        vertical_win = [True for _ in range(self.N)]
        horizontal_win = [True for _ in range(self.N)]
        diagonal_win = [True for _ in range(2)]
        for h in range(self.N):
            for w in range(self.N):
                # 가로, 세로
                if self.map[h][w] != current_player_type:
                    vertical_win[h] = False
                    horizontal_win[w] = False
                else:
                    pass
                # 왼 대각
                if h == w and self.map[h][w] != current_player_type:
                    diagonal_win[0] = False
                # 오른 대각
                rotated_w = abs(w - (self.N - 1))
                if h == rotated_w and self.map[h][w] != current_player_type:
                    diagonal_win[1] = False
        if any(vertical_win) or any(horizontal_win) or any(diagonal_win):
            return True
        else:
            return False

    def is_full(self):
        for h in range(self.N):
            for w in range(self.N):
                if self.map[h][w] == 'E':
                    return False
                else:
                    pass
        return True

    def print_description(self):
        print("** Initial NxN Tic-tac-toe Map **")
        self.print_current_map()

        print("** Action Indexes **")
        for idx, des in enumerate(self.map_index_description):
            print(des, end=' ')
            if (idx + 1) % self.N == 0:
                print('\n', end='')

    def print_current_map(self):
        for h in range(self.N):
            for w in range(self.N):
                print(self.map[h][w], end=' ')
            print('\n', end='')
        print()

    # Fill this function
    def match_prediction(self):
        # x : 현재 플레이어
        # return value (이긴다 : 1 무승부 : 0 진다 : -1)
        # print("현재 플레이어의 최대 결과는 승리입니다.") -> example
        # print("현재 착수할 플레이어가 이긴다고 확신이 드는가? Yes or No")
        if self.is_win(self.player_types[(1 - self.global_step) % 2]): return -1
        
        prediction = 2
        for h in range(self.N):
            for w in range(self.N):
                if self.map[h][w] == 'E' :
                    self.map[h][w] = self.player_types[self.global_step % 2]
                    self.global_step += 1
                    prediction = min(prediction, self.match_prediction())
                    self.global_step -= 1
                    self.map[h][w] = 'E'
        
        if prediction == 2:
            prediction = 0

        return -prediction


if __name__ == '__main__':
    game = TicTacToe()
    game.print_description()

    game.reset()
    done = False
    while not done:
        print()

        print(game.player_types[game.global_step % 2], "의 차례입니다.", sep='')
        
        # Do it.
        # game.match_prediction()
        prediction = game.match_prediction()
        if(prediction == 1):
            print(game.player_types[game.global_step % 2], "의 최대 결과는 승리입니다.", sep='')
        elif(prediction == 0):
            print(game.player_types[game.global_step % 2], "의 최대 결과는 무승부입니다.", sep='')
        elif(prediction == -1):
            print(game.player_types[game.global_step % 2], "의 최대 결과는 패배입니다.", sep='')

        action = int(input('Select action please: '))
        if not(game.map_index_description[0] <= action <= game.map_index_description[-1]):
            done = True
            print("Error: You entered the wrong number.")
            continue
        _, player_result, done = game.step(action)
        game.print_current_map()
        if done:
            for player, result in player_result.items():
                if result == game.win_reward:
                    player_result[player] = 'win'
                elif result == game.defeat_reward:
                    player_result[player] = 'defeat'
                else:
                    player_result[player] = 'draw'
            print(player_result)


** Initial NxN Tic-tac-toe Map **
E E E 
E E E 
E E E 

** Action Indexes **
0 1 2 
3 4 5 
6 7 8 

X의 차례입니다.
X의 최대 결과는 무승부입니다.
Select action please: 4
E E E 
E X E 
E E E 


O의 차례입니다.
O의 최대 결과는 무승부입니다.
Select action please: 2
E E O 
E X E 
E E E 


X의 차례입니다.
X의 최대 결과는 무승부입니다.
Select action please: 6
E E O 
E X E 
X E E 


O의 차례입니다.
O의 최대 결과는 무승부입니다.
Select action please: 1
E O O 
E X E 
X E E 


X의 차례입니다.
X의 최대 결과는 승리입니다.
Select action please: 0
X O O 
E X E 
X E E 


O의 차례입니다.
O의 최대 결과는 패배입니다.
Select action please: 3
X O O 
O X E 
X E E 


X의 차례입니다.
X의 최대 결과는 승리입니다.
Select action please: 8
X O O 
O X E 
X E X 

{'X': 'win', 'O': 'defeat'}
